In [20]:
from PIL import Image
import numpy as np
import math
import random
import pandas as pd
import webcolors
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

FOLDER_PATH = 'data/'

In [21]:
def load_image(filename: str, resize: tuple):
  img = Image.open(filename)
  img = img.resize(resize)
  return np.array(img).astype(np.float64)

In [22]:
def show_image(filename: str):
  img = Image.open(filename)
  img.show()

In [23]:
def euclidean_distance(p1, p2):
    return math.sqrt(sum((px - qx) ** 2.0 for px, qx in zip(p1, p2)))

In [24]:
def manhattan_distance(p1, p2):
  return sum(abs(px - qx) for px, qx in zip(p1, p2))

In [25]:
def nearest_centroid(point, centroids, distance):
  flag = False
  idx_centroid, min_distance = 0, 0

  for i in range(len(centroids)):
    current_distance = distance(point, centroids[i])
    if not flag or current_distance <  min_distance:
      flag = True
      
      idx_centroid = i
      min_distance = current_distance

  return idx_centroid, min_distance

In [26]:
def lloyd(data, k, iters, type, distance):
  distance_func = euclidean_distance if distance == "euclidian" else manhattan_distance

  go = True
  epoch_c = 0
  error = 0

  cluster_array = np.zeros(len(data))

  centroids = random.choices(data, k=k)

  while go:
    for element, i in zip(data, range(len(cluster_array))):
      idx_centroid, min_distace = nearest_centroid(
        element, centroids, distance_func)

      error += min_distace
      cluster_array[i] = idx_centroid

    for i in range(k):
      cluster_average = []
      for element, j in zip(data, range(len(cluster_array))):
        if cluster_array[j] == i:
          cluster_average.append(element)

      centroids[i] = np.average(cluster_average, axis=0)
      
    epoch_c += 1
    go = False if epoch_c > iters else True

    return centroids, error



In [27]:
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

In [28]:
def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

In [29]:
img = "img_01.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'saddlebrown')
(None, 'chocolate')
(None, 'maroon')
(None, 'chocolate')
(None, 'chocolate')


In [30]:
color_palletes

[array([171.89509306,  75.38832487,   3.32318105]),
 array([196.74247492, 101.67409885,   4.35377183]),
 array([121.96715873,  34.00845911,   2.28396086]),
 array([213.72564319, 122.86482497,   5.1480388 ]),
 array([187.2078853 ,  94.55913978,   3.7562724 ])]

In [31]:
img = "img_02.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'indigo')
(None, 'purple')
(None, 'mediumpurple')
(None, 'darkslategray')
(None, 'darkslategray')


In [32]:
color_palletes

[array([85.40792596, 35.11888942, 90.64095871]),
 array([117.73229706,  38.34838391, 138.26745621]),
 array([149.15633737, 101.9361603 , 171.44967381]),
 array([61.55696203, 25.22580645, 64.42792977]),
 array([49.61046512, 23.80377907, 53.52979651])]

In [33]:
img = "img_03.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

ValueError: cannot reshape array of size 49152 into shape (16384,)

In [ ]:
color_palletes


[array([115.67199148, 104.29925453,  72.09052183]),
 array([133.44477172, 123.87223859,  87.87849779]),
 array([181.16311056, 182.59143008, 157.22024334]),
 array([149.06285907, 142.36633998, 111.80770531]),
 array([75.43669188, 58.80946572, 36.19492559])]

In [ ]:
img = "img_04.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'saddlebrown')
(None, 'maroon')
(None, 'lightslategray')
(None, 'dimgray')
(None, 'dimgray')


In [ ]:
color_palletes


[array([112.89376915,  46.81001021,  56.30643514]),
 array([83.8847481 , 20.82125604, 25.4478951 ]),
 array([140.76181818, 132.85972028, 148.59062937]),
 array([92.43692308, 65.52615385, 85.33923077]),
 array([122.71195039,  82.64656144, 102.61696731])]

In [ ]:
img = "img_05.png"
show_image(FOLDER_PATH + img)

img_array = load_image(FOLDER_PATH + img, (128, 128))
img_array = img_array.reshape(128*128, 3)

color_palletes, error = lloyd(img_array, 5, 1e5, "", "euclidean")

for color in color_palletes:
  print(get_colour_name(color))

(None, 'black')
(None, 'dimgray')
(None, 'darkslategray')
(None, 'darkslategray')
(None, 'black')


In [ ]:
color_palletes


[array([ 3.65859684, 14.38290514, 24.82608696]),
 array([112.4665528 , 113.57216055, 115.24195844]),
 array([ 5.69950739, 50.29474548, 51.11083744]),
 array([55.22880332, 56.31497151, 54.78466586]),
 array([21.56449835, 25.71168688, 29.44266814])]